## Spark
---
#### Elo notes

In [3]:
import string
import json
import pickle as pk
import pandas as pd
import numpy as np

from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re

from collections import Counter


In [4]:
from pyspark.sql import SQLContext

In [132]:
from pyspark.sql import Row

In [5]:
from sklearn.datasets import fetch_20newsgroups

[20 Newsgroups](http://qwone.com/~jason/20Newsgroups/)

In [6]:
sc

In [7]:
news_ = fetch_20newsgroups(subset='train')

In [8]:
categories = news_.target_names

In [9]:
categories

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [10]:
news = news_.data

In [11]:
news[2]

u'From: twillis@ec.ecn.purdue.edu (Thomas E Willis)\nSubject: PB questions...\nOrganization: Purdue University Engineering Computer Network\nDistribution: usa\nLines: 36\n\nwell folks, my mac plus finally gave up the ghost this weekend after\nstarting life as a 512k way back in 1985.  sooo, i\'m in the market for a\nnew machine a bit sooner than i intended to be...\n\ni\'m looking into picking up a powerbook 160 or maybe 180 and have a bunch\nof questions that (hopefully) somebody can answer:\n\n* does anybody know any dirt on when the next round of powerbook\nintroductions are expected?  i\'d heard the 185c was supposed to make an\nappearence "this summer" but haven\'t heard anymore on it - and since i\ndon\'t have access to macleak, i was wondering if anybody out there had\nmore info...\n\n* has anybody heard rumors about price drops to the powerbook line like the\nones the duo\'s just went through recently?\n\n* what\'s the impression of the display on the 180?  i could probably swi


[from Sklearn to MLlib](http://blog.xebia.fr/2015/10/08/from-scikit-learn-to-spark-ml/)


In [13]:
sc

In [14]:
sqlCtx = SQLContext(sc)

In [15]:
sqlCtx = categories

In [17]:
# Raw data
text_rdd = sc.wholeTextFiles('file:///Users/eloisaelias/../20_newsgroups/*').cache()

In [18]:
text_rdd.take(3)

[(u'file:/Users/eloisaelias/Desktop/weeks/20_newsgroups/talk.politics.mideast/75895',
  u'Newsgroups: talk.politics.mideast\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!uunet!brunix!doorknob!hm\nFrom: hm@cs.brown.edu (Harry Mamaysky)\nSubject: Heil Hernlem \nIn-Reply-To: hernlem@chess.ncsu.edu\'s message of Wed, 14 Apr 1993 12:58:13 GMT\nMessage-ID: <HM.93Apr15112701@yoda.cs.brown.edu>\nSender: news@cs.brown.edu\nOrganization: Dept. of Computer Science, Brown University\nReferences: <1993Apr14.125813.21737@ncsu.edu>\nDate: Thu, 15 Apr 1993 16:27:01 GMT\nLines: 24\n\nIn article <1993Apr14.125813.21737@ncsu.edu> hernlem@chess.ncsu.edu (Brad Hernlem) writes:\n\n   Lebanese resistance forces detonated a bomb under an Israeli occupation\n   patrol in Lebanese territory two days ago. Three soldiers were killed and\n   two wounded. In "retaliation", Israeli and Israeli-backed forces wou

#### elo, check page 66

get the filepath

[I have to stackoverflow things](https://stackoverflow.com/questions/43209811/pyspark-how-to-access-the-values-in-a-tuple-in-a-key-tuple-rdd-python)

In [142]:
import pyspark.sql.functions as sf

In [20]:
import pyspark.sql.types as sparktypes

In [84]:
from pyspark.sql.functions import col

In [146]:
# from pyspark.sql.functions import substring_index

In [21]:
sc

In [22]:
sqlc =  SQLContext(sc)

In [23]:
df = sqlc.createDataFrame(text_rdd, ['filepath', 'text'])

In [24]:
df.show()

+--------------------+--------------------+
|            filepath|                text|
+--------------------+--------------------+
|file:/Users/elois...|Newsgroups: talk....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Path: cantaloupe....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Path: cantaloupe....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Path: cantaloupe....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Path: cantaloupe....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Newsgroups: talk....|
|file:/Users/elois...|Newsgroups: talk....|
|file:/Users/elois...|Newsgroups: talk....|
|file:/Users/elois...|Xref: cantaloupe....|
|file:/Users/elois...|Newsgroups: talk....|
|file:/Users/elois...|Xref: cant

In [25]:
filepath = df.select('filepath')

In [27]:
filepath.show()

+--------------------+
|            filepath|
+--------------------+
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
|file:/Users/elois...|
+--------------------+



In [28]:
text = df.select('text')

In [29]:
text.show()

+--------------------+
|                text|
+--------------------+
|Newsgroups: talk....|
|Xref: cantaloupe....|
|Xref: cantaloupe....|
|Path: cantaloupe....|
|Xref: cantaloupe....|
|Path: cantaloupe....|
|Xref: cantaloupe....|
|Xref: cantaloupe....|
|Path: cantaloupe....|
|Xref: cantaloupe....|
|Xref: cantaloupe....|
|Xref: cantaloupe....|
|Path: cantaloupe....|
|Xref: cantaloupe....|
|Newsgroups: talk....|
|Newsgroups: talk....|
|Newsgroups: talk....|
|Xref: cantaloupe....|
|Newsgroups: talk....|
|Xref: cantaloupe....|
+--------------------+



#### Extracting IDs
[I have to stackoverflow things](https://forums.databricks.com/questions/1056/access-to-column-value-of-a-row.html)

In [105]:
filepath

DataFrame[filepath: string]

In [147]:
renglon = filepath.first()